# ARIMAX

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import statslib as stb

import statsmodels.api as sm

import pandas as pd
from copy import deepcopy
import matplotlib as mpl
import matplotlib.pyplot as plt

figsize = (8*1.6, 8)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Data Set

In [ ]:
stb.datasets.uschange.df.head()

In [ ]:
y = stb.datasets.uschange.df.Consumption

In [ ]:
X = stb.datasets.uschange.df.drop('Consumption', axis=1)

# Design Matrix

In [ ]:
DM = stb.DesignMatrix(y, X)

In [ ]:
DM.describe()

In [ ]:
DM.names

In [ ]:
DM.plot()

In [ ]:
DM.plot_scatter_lowess()

# Grid Search

When fittind models, it is possible to increase the likelihood by adding parameters, but doing so may resut in overfitting. Both AIC, AICc and BIC attempt to resolve this problem by introducing a penalty term for the number of parameters in the model; the penalty term is larger in BIC than in AIC

## Information criterions

Likelihood function:
$$
L = p (x | \hat{\theta}, Model)
$$

### Akaike information criterion
$$
AIC = 2k - 2\ln(L)
$$


### Akaike with correction for small sample sizes

$$
AICc = AIC + \dfrac{2k^2 + 2k}{n-k-1}
$$

$ \\ $

*Hirotugu Akaike 1971*

### Bayesian information criterion

$$
BIC = k \ln(n) - 2\ln(L)
$$


where 
* $L$ - is maximum of the likelihood function
* $k$ - number of parameters estimated
* $n$ - number of data points

In [ ]:
from itertools import product

grid = product(range(1,3), [0], range(1,3))

results = []
for elem in grid:
    try:
        gc = stb.GeneralCalibrator(sm.tsa.arima.ARIMA, 
                                       dict(order=elem, 
                                       trend='n', 
                                       enforce_stationarity=True))
        gm = stb.GeneralModel(gc, DM)
        gm.fit(range(10))
        results.append([elem, gm.fitted.aic])
    except:
        continue

pd.DataFrame(results, columns=['elem', 'metric']).sort_values(by=['metric'], ascending=True)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=6)
splits = [(train_idx, test_idx) for train_idx, test_idx in tscv.split(DM.dm.index)]

In [ ]:
metric = stb.metrics.mean_absolute_percentage_error

In [ ]:
gc = stb.GeneralCalibrator(sm.tsa.arima.ARIMA, 
                               dict(order=(2,0,1), 
                               trend='n', 
                               enforce_stationarity=True))
gm = stb.GeneralModel(gc, DM)
cv = stb.CrossValidation(gm, splits, metric=metric)
cv_m, cv_std = cv.run()

# Final Model

In [ ]:
gc = stb.GeneralCalibrator(sm.tsa.arima.ARIMA, 
                               dict(order=(2,0,1), 
                               trend='n', 
                               enforce_stationarity=True))
gm = stb.GeneralModel(gc, DM)

gm.fit(range(DM.n))

gm.forecast(range(DM.n))

print(gm.fitted.summary())

In [ ]:
gm.plot_diagnostics()

**Jarque Bera test for 3d and 4th moment matching to the Normal one:**

In [ ]:
stb.stat_tests.test_jarque_bera(gm.std_residuals)

**Breusch Pagan Test for homoscedasticity**

In [ ]:
stb.stat_tests.test_breusch_pagan(gm.residuals, gm.fitted.model.exog)

# Diagnostics

In [ ]:
gm.plot_diagnostics()

**Jarque Bera test for 3d and 4th moment matching to the Normal one:**

In [ ]:
stb.stat_tests.test_jarque_bera(gm.std_residuals)

**Breusch Pagan Test for homoscedasticity**

In [ ]:
stb.stat_tests.test_breusch_pagan(gm.residuals, gm.fitted.model.exog)